In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import pickle

Read in the pickle file and find the top five occurance rate payloads in the file
Return the payloads as 'text'讀取出text 

In [ ]:
with open("pop_report_with_tknscore_new.pkl", "rb") as f:
    pop_df = pickle.load(f)

with open("imap_report_with_tknscore_new.pkl", "rb") as f:
    imap_df = pickle.load(f)

with open("smtp_report_with_tknscore_new.pkl", "rb") as f:
    smtp_df = pickle.load(f)

with open("sip_report_with_tknscore_new.pkl", "rb") as f:
    sip_df = pickle.load(f)

label_df = pd.read_csv("sid_table(packet).csv")

## 流程
### 獲取目標資料
- 獲取特定資料集的所有SID(unique)
- 獲取各個SID所對應到的text
- 過濾出sizt(text)>5的SID
- 對這些SID用LLM自動生成regex(生成Regex流程)
### 生成Regex
- 對每一個SID給LLM幾個regex例子
- 要求LLM生成三個不一樣但是可以過濾text的regex
### 測試Regex
- 拿答案給的regex和我們生成的regex測試能否的到相同結果(2/3過算對)

In [ ]:
#獲取特定資料集的所有SID
def get_unique_sids_list(df):
    unique_sids = df[df['text'].notnull()]['sid'].unique()
    return unique_sids
sidList = get_unique_sids_list(pop_df)
print(sidList)
print(len(sidList))

In [ ]:
#獲取各個SID所對應到的text
def map_sid_to_unique_texts(df):
    sid_to_texts = {}
    for _, row in df.iterrows():
        sid = row['sid']
        text = row['text']
        if pd.notnull(text):  # Ensure text is not NaN
            if sid in sid_to_texts:
                sid_to_texts[sid].add(text)
            else:
                sid_to_texts[sid] = {text}
    return sid_to_texts
sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
print(sid_to_unique_texts)

In [ ]:
#過濾掉text長度小於5的sid
def filter_sids_by_text_length(sit_to_text_dict):
    filtered_sids = set()
    for key, value in sit_to_text_dict.items():
        if len(value) > 5:  # Convert to string to avoid errors with non-string types
            filtered_sids.add(key)
    return filtered_sids

# Assuming pop_df is your DataFrame
filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
print(filtered_sids)
print(len(filtered_sids))

### ChatGPT API


In [ ]:
load_dotenv()
CHAT_GPT_API_KEY = os.getenv('C_API_KEY')
client = OpenAI(
    api_key=CHAT_GPT_API_KEY
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


### Gemini API

In [ ]:
load_dotenv()

system_prompt = "You are a cat. Your name is Neko."
user_prompt = "What is your name?"
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=system_prompt)

response = model.generate_content(user_prompt)
print(response.text)

### Evaluation Generated Regex
Randonly take 20 positive and 20 negateve data as input to generated regex and answer regex, comparing there result.

In [65]:

import re
import csv
import random

#隨機選擇除了給定的SID以外的100個text
def select_random_texts(sid_to_unique_texts, given_sid, num_texts=100):
    # Filter out the given SID
    filtered_texts = [texts for sid, texts in sid_to_unique_texts.items() if sid != given_sid]
    
    # Flatten the list of lists to a single list of texts
    all_texts = [text for sublist in filtered_texts for text in sublist]
    
    # Randomly select 100 texts, or all texts if there are fewer than 100
    selected_texts = random.sample(all_texts, min(len(all_texts), num_texts))
    
    return selected_texts

def get_pcre_by_sid(sid):
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['SID'] == str(sid):
                # Remove leading and trailing slashes from the pcre value
                return row['pcre'].strip('/')
    return None

def match_patterns(targetText, threeAnsPattern,ansPattern):
    match1 = bool(re.search(threeAnsPattern[0], targetText))
    match2 = bool(re.search(threeAnsPattern[1], targetText))
    match3 = bool(re.search(threeAnsPattern[2], targetText))
    match4 = bool(re.search(threeAnsPattern[3], targetText))
    match5 = bool(re.search(threeAnsPattern[4], targetText))
    match6 = bool(re.search(threeAnsPattern[5], targetText))
    match7 = bool(re.search(threeAnsPattern[6], targetText))
    match8 = bool(re.search(threeAnsPattern[7], targetText))
    match9 = bool(re.search(threeAnsPattern[8], targetText))
    ourResult = sum([match1, match2, match3, match4, match5, match6, match7, match8, match9]) >= 7
    ansMatch = bool(re.search(ansPattern, targetText))
    return ourResult == ansMatch
def positive_evaluation(threeAnsPattern,sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + match_patterns(text, threeAnsPattern, ansPattern)
        if not match_patterns(text, threeAnsPattern, ansPattern):
            errorList.append(text)

    print("positive test")
    print(f"correct: {correct}, total: {total}")
    print(f"errorList: {errorList}")
    return correct, total

def negative_evaluation(GeneratedPatternList,sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    texts = select_random_texts(sid_to_unique_texts, sid)
    for text in texts:
        total = total + 1
        correct = correct + match_patterns(text, GeneratedPatternList, ansPattern)
        if  not match_patterns(text, GeneratedPatternList, ansPattern):
            errorList.append(text)
    print("negative test")
    print(f"correct: {correct}, total: {total}")
    for error_text in errorList:
        print(error_text)
    #print(f"errorList: {errorList}")
GeneratedPatternList = [
    r"^USER [^\s@]+(?:@[^\s@]+\.[^\s@]+)?\r\n$",
    r"^USER (anonymous|[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})\r\n$",
    r"^USER (spam\.(blocked|passed)@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}|anonymous)\r\n$",
    r"^USER (\S+@\S+\.\S+|\S+)\r\n$",
    r"^USER \S+@\S+\.\S+\r\n$|^USER \S+\r\n$",
    r"^USER (\w+@\w+\.\w+|\w+)\r\n$",
    r"^USER\s\S+(\.\S+)?\r\n$",
    r"^USER\s[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}\r\n|^USER\sanonymous\r\n$",
    r"^USER\s([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}|anonymous)\r\n$"
]

sid = '1648627'
positive_evaluation(GeneratedPatternList, sid)
negative_evaluation(GeneratedPatternList, sid)

positive test
correct: 25, total: 45
errorList: ['USER administrator\r\n', 'USER root\r\n', 'USER test\r\n', 'USER data\r\n', 'USER pwrchute\r\n', 'USER informix\r\n', 'USER oracle\r\n', 'USER access\r\n', 'USER admin\r\n', 'USER account\r\n', 'USER server\r\n', 'USER webmaster\r\n', 'USER user\r\n', 'USER backup\r\n', 'USER www\r\n', 'USER lizdy\r\n', 'USER sybase\r\n', 'USER web\r\n', 'USER oracle8\r\n', 'USER world@gmail.com']
negative test
correct: 100, total: 100


### Helper Function

In [ ]:
def list_csv_columns():
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        # Print all column names
        print(reader.fieldnames)

In [ ]:
#print filtered_sids and corresponding text
for sid in filtered_sids:
    print(f"sid: {sid}, text: {sid_to_unique_texts[sid]}")

In [ ]:
import random 

protocols = [
    {'name': 'pop_df', 'data': pop_df},
    {'name': 'imap_df', 'data': imap_df},
    {'name': 'smtp_df', 'data': smtp_df},
    {'name': 'sip_df', 'data': sip_df}
]

with open('target.txt', 'w') as file:
    pass

with open('target.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in filtered_sids:
            for i in range (3):
                texts = list(sid_to_unique_texts[sid])
                random.shuffle(texts)
                selected_texts = texts[:5]
                file.write(f"sid: {sid}, text: {selected_texts}\n")
        file.write("\n")

In [ ]:
with open('all_texts.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in filtered_sids:
            file.write(f"sid: {sid}, text: {sid_to_unique_texts[sid]}\n")

        file.write("\n")